<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/myStocks_Portfolio_Analysis_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
import os

# Install es_MX
!/usr/share/locales/install-language-pack en_IN
!dpkg-reconfigure locales

# Restart Python process to pick up the new locales
os.kill(os.getpid(), 9)
'''

'\nimport os\n\n# Install es_MX\n!/usr/share/locales/install-language-pack en_IN\n!dpkg-reconfigure locales\n\n# Restart Python process to pick up the new locales\nos.kill(os.getpid(), 9)\n'

In [2]:
!pip install pyxirr

In [3]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import os
project_path = '/content/drive/My Drive/tradevesting'
os.chdir(project_path)

import warnings
warnings.filterwarnings('ignore')

from google.colab import data_table

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Import the function from the script
import myStocks_Portfolio_Analysis_v2

# Call the function to get the DataFrame
df_portfolio_features = myStocks_Portfolio_Analysis_v2.get_portfolio_features_v2()

cols = ['Symbol', 'Close', 'AvgCost','FTT',  'Dev%_200', 'Dev%_PE', 'Momentum', 'Shares',  'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%','XIRR%', 'CurrAlloc%', 'RRR Ind', 'Criteria', 'Strategy', 'Risk Ind', 'Gained%', 'LatestQtr', 'StarStock', 'Conviction', 'Category']
df_portfolio_features = df_portfolio_features[cols]

Run date time (IST): 2024-07-19 15:24:28
qualified stocks: 79
with latest results: 78
still star stocks: 46
-------------------
Initial Investment:  ₹86,46,489.00
CY Investment:  ₹99,57,489.00
Reserve:  ₹1,63,480.00
Current:  ₹1,07,86,751.00
-------------------
Today PnL: -₹1,84,176.00 (-1.68%)
Current PnL: ₹4,44,262.00 (4.46%)
CY Booked + Current PnL: ₹8,29,262.00 (8.33%)
-------------------
Total profit:  ₹8,76,332.00
Total loss:  -₹4,32,069.00
-------------------
Total Booked + Current PnL: ₹21,40,262.00 (24.75%)
Total Booked PnL: ₹16,96,000.00 (19.61%)
Curr Year Booked PnL: ₹3,85,000.00 (3.57%)
Prev Year Booked PnL: ₹13,11,000.00 (15.16%)
Est NTT:  ₹1,50,20,413.00
Est NTT PnL: ₹42,33,662.00 (39.25%)
-------------------
Est LTT:  ₹2,01,75,888.00
Est LTT PnL: ₹93,89,137.00 (87.04%)
Deployed:  ₹86,02,000.00
Current:  ₹1,07,86,751.00
CAGR/XIRR %: 26.06%


In [5]:
# all portfolio
df_portfolio_features = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
data_table.DataTable(df_portfolio_features, include_index=False)

,Symbol,Close,AvgCost,FTT,Dev%_200,Dev%_PE,Momentum,Shares,Current,Current P/L,...,CurrAlloc%,RRR Ind,Criteria,Strategy,Risk Ind,Gained%,LatestQtr,StarStock,Conviction,Category
18,CAMS,4136.30,2424.02,4450.00,38.90,31.61,UP,25,103408.0,42808.0,...,0.97,5.46,AVR,BTT,69.0,87.10,1,1,M,MISC
48,MARICO,668.05,530.08,725.00,22.13,16.59,UP,200,133610.0,27594.0,...,1.26,2.42,AV40,BTT,33.0,36.32,1,1,H,FMCG
62,SHALBY,283.40,264.74,327.00,1.58,NaN,UP,302,85587.0,5636.0,...,0.81,0.43,AVSP,NTT,6.0,53.87,1,1,H,HEALTHCARE
21,DABUR,632.40,523.35,710.00,15.82,10.55,UP,175,110670.0,19084.0,...,1.04,1.41,AV40,BTT,22.0,28.25,1,1,H,FMCG
29,HCLTECH,1596.55,1349.00,1821.00,12.20,34.77,UP,75,119741.0,18566.0,...,1.13,1.10,AV40,BTT,21.0,49.16,1,1,H,IT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,AAVAS,1739.00,1840.86,3195.80,10.83,-39.21,UP,89,154771.0,-9066.0,...,1.46,-0.07,AVSP,ATH,-8.0,32.18,1,1,M,FINANCE
30,HDFCBANK,1606.95,1569.25,2170.00,5.72,-10.87,NaN,272,437090.0,10254.0,...,4.11,0.07,AV40,BTT,10.0,17.69,1,1,H,BANKS
68,TANLA,967.80,969.42,2007.31,0.39,-18.81,DOWN,166,160655.0,-269.0,...,1.51,-0.00,AAR,ATH,-0.0,21.02,1,1,L,IT
9,BANDHANBNK,191.21,227.40,400.00,-9.38,-26.83,NaN,827,158131.0,-29929.0,...,1.49,-0.17,AVSP,NTT,-24.0,10.33,1,0,H,BANKS
